Input/output
============

HDF5 using silx
===============

Read example
------------

In [ ]:
import silx.io.utils

h5file = silx.io.utils.load('data/test.h5')

# print available names at the first level
print(h5file['/'].keys())

# reaching a dataset from a sub group
dataset = h5file['/diff_map_0000/data/map']

# using size and types to not read the full stored data
print(dataset.shape, dataset.size, dataset.dtype)

# datasets mimics numpy-array
# read and apply the operation
a = 2 * dataset[0, 5]
# copy the data and store it as a numpy-array
b = dataset[...]


Write example
-------------

In [ ]:
import numpy
import h5py

data = numpy.arange(10000.0)
data.shape = 100, 100

# write
h5file = h5py.File('my_first_one.h5', access='w')

# write data into a dataset from the root
h5file['/data1'] = data

# write data into a dataset from group1
h5file['/group1/data2'] = data

h5file.close()

Specfile using silx
===================

HDF5-like mapping
-----------------

In [ ]:
import silx.io.utils

h5file = silx.io.utils.load('data/test.h5')

string = silx.io.utils.h5ls(h5file)
print(string)

Python example
--------------

In [ ]:
import silx.io.utils

specdata = silx.io.utils.load('data/oleg.dat')

# print available scans
print(specdata['/'].keys())

# print available measurements from the scan 94.1
print(specdata['/94.1/measurement'].keys())

# get data from measurement
xdata = specdata['/94.1/measurement/Epoch']
ydata = specdata['/94.1/measurement/bpmi']

EDF using FabIO
===============

Reading files
-------------

In [ ]:
import fabio

image = fabio.open("data/medipix.edf")

# here is the data as a numpy array
image.data

# here is the header as key-value dictionary
image.header


Writing files
-------------

In [ ]:
import fabio
import numpy

image = numpy.random.rand(10, 10)
metadata = {'pixel_size': '0.2'}

image = fabio.edfimage.edfimage(data=image, header=metadata)
image.write('new.edf')

File conversion
===============

With FabIO
---------------------

In [ ]:
import fabio

image = fabio.open('data/medipix.edf')
image = image.convert('tif')
image.save('filename.tif')

In [ ]:
ls -al filename.*

Spec file to HDF5
-----------------

In [ ]:
from silx.io.spectoh5 import write_spec_to_h5

write_spec_to_h5('data/oleg.dat', 'oleg.h5', mode='w')

In [ ]:
ls -al oleg.*

Exercise
========

1. Read the EDF file ``medipix.edf``.
2. Process the data

   - Create a mask for all the values below 10%.
   - With the above mask, set the affected pixels to 10%.
   - Optionally do the same for values above 90%.
   - This clamp values between 10% and 90%

3. Store the source, the mask of changed pixels and the result inside ``process.h5``, as below.

   ![Output file structure](images/exercise-result.png)

4. Load ``process.h5`` and list the root content


In [ ]:
# Load data/medipix.edf
# ...

# Process the data
# ...

# Load process.h5 to write inside: the source, the mask and the result
# ...